# Exercice 8
## Utiliser des APIs
### Consigne
* Récupérer l'item policy d'un exemplaire 
* Modifier l'item policy de l'exemplaire vers 21 (2 semaines)

**Exercice avancé:** 
* Récupérer le user group pour un ensemble d'utilisateurs

In [ ]:
# Importer les bibliothèques requises
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import requests
import json

PATH_TO_SECRETS = '../Secrets/'

In [ ]:
# Chargement des données
df = pd.read_csv('Data/barcode_hph_sb.csv')

In [ ]:
df.head()

### Récupérer l'item policy
Documentation Alma:
* Récupérer les données d'un exemplaire: https://developers.exlibrisgroup.com/alma/apis/docs/bibs/R0VUIC9hbG1hd3MvdjEvYmlicy97bW1zX2lkfS9ob2xkaW5ncy97aG9sZGluZ19pZH0vaXRlbXMve2l0ZW1fcGlkfQ==/
* Modifier un exemplaire: https://developers.exlibrisgroup.com/alma/apis/docs/bibs/UFVUIC9hbG1hd3MvdjEvYmlicy97bW1zX2lkfS9ob2xkaW5ncy97aG9sZGluZ19pZH0vaXRlbXMve2l0ZW1fcGlkfQ==/
* Récupérer les données d'une holding: https://developers.exlibrisgroup.com/alma/apis/docs/bibs/R0VUIC9hbG1hd3MvdjEvYmlicy97bW1zX2lkfS9ob2xkaW5ncy97aG9sZGluZ19pZH0=/
* Modifier une holding: https://developers.exlibrisgroup.com/alma/apis/docs/bibs/UFVUIC9hbG1hd3MvdjEvYmlicy97bW1zX2lkfS9ob2xkaW5ncy97aG9sZGluZ19pZH0=/

In [ ]:
# Clé Sandbox en lecture et écriture pour les ressources bibliographiques
with open(PATH_TO_SECRETS + 'API_KEY_ITEMS_SB') as f:
    # Il faut utiliser "strip" pour supprimer le saut de ligne à la fin
    API_KEY_BIB_SB = f.read().strip()

# URL de base de l'api des utilisateurs dans Alma
API_ITEM_URL = 'https://api-eu.hosted.exlibrisgroup.com/almaws/v1/items'

# En-têtes de la future requête
# Définit notamment le format de la réponse
HEADERS = {'content-type': 'application/json',
           'accept': 'application/json',
           'Authorization': 'apikey ' + API_KEY_BIB_SB}

In [ ]:
# Récupération de la cote en fonction du code-barre
barcode = 'HR40042121'
r1 = requests.get(API_ITEM_URL, params={'item_barcode': barcode}, headers=HEADERS)

data1 = r1.json()
data1['item_data']['policy']

### Modifier l'item policy
Note: le plus simple est de remplacer le dictionnaire de l'item policy par un nouveau qui ne comporte que
"value". Le champ "desc" est facultatif pour effectuer une modification.

In [ ]:
data1['item_data']['policy'] = {'value': '21'}
url = data1['link']

In [ ]:
r2 = requests.put(url, headers=HEADERS, data=json.dumps(data1))
data2 = r2.json()
data2['item_data']['policy']

## Exercice avancé
Le but de cet exercice est d'interroger l'API swisscovery pour récupérer tous les user groups
des participants du cours.

Quand il ne sont pas inscris, afficher "NO ACCOUNT".

La liste des participants se trouve dans les "Secrets".

La documentation de l'API se trouve ici: https://developers.exlibrisgroup.com/alma/apis/users/

Pour réussir cette opération, il faut effectuer deux requêtes:
1. Rechercher l'utilisateur par email: https://developers.exlibrisgroup.com/alma/apis/docs/users/R0VUIC9hbG1hd3MvdjEvdXNlcnM=/
2. Recupérer le primary_id et rechercher les détails de l'utilisateur: https://developers.exlibrisgroup.com/alma/apis/docs/users/R0VUIC9hbG1hd3MvdjEvdXNlcnMve3VzZXJfaWR9/
3. Récupérer le user_group

### Solution simple avec "apply"
Créer une fonction qui récupère le user_group à partir de l'email
puis créer une nouvelle colonne en utilisant "apply"

In [ ]:
# Créer un DataFrame en récupérant les la liste des participants dans les secrets
df = pd.read_csv(PATH_TO_SECRETS + 'participants.csv')

In [ ]:
# Clé NZ en lecture pour les utilisateurs
with open(PATH_TO_SECRETS + 'API_KEY_USERS_NZ') as f:
    API_KEY_USERS_NZ = f.read().strip()

# URL de base de l'api des utilisateurs dans Alma
API_BASE_URL = 'https://api-eu.hosted.exlibrisgroup.com/almaws/v1/users/'

# En-têtes de la future requête
# Définit notamment le format de la réponse
HEADERS = {'content-type': 'application/json',
           'accept': 'application/json',
           'Authorization': 'apikey ' + API_KEY_USERS_NZ}

def get_user_group_from_email(email):
    # Il faut interroger l'API en faisant une requête avec le paramètre "q"
    r1 = requests.get(API_BASE_URL, params={'q': f'email~{email}'}, headers=HEADERS)
    
    # r1.ok vaut True si la requête a retourné un code de réponse inférieur à 400
    if r1.ok is not True:
        return 'ERROR'
    
    data = r1.json()
    
    # S'il n'y a pas d'utilisateur, il n'y aura pas de clé "user".
    # Une exception "KeyError" sera générée et interceptée
    try:
        link = data['user'][0]['link']
    except KeyError:
        return 'NO ACCOUNT'
    
    r2 = requests.get(link, headers=HEADERS)
    if r2.ok is not True:
        return 'ERROR'
    
    try:
        user_group = r2.json()['user_group']['desc'].strip()
        
    except KeyError:
        user_group = ''
    
    return user_group

# Crée une colonne à l'aide d'apply
df['User group'] = df['Email'].apply(get_user_group_from_email)

**Remarque importante**
Ne jamais utiliser `apply` ainsi sur de grands sets de donnéespour
faire des appels d'APIs.

Si le programme plante, tout le traitement serait perdu.

Si le set n'est pas trop grand, on peut itérer avec `iterrows` et
enregistrer les résultats au fur et à mesure dans un nouveau
DataFrame.

### Solution plus complexe avec "iterrows"
Le but cette fois-ci est d'enregistrer un nouveau DataFrame après le traitement
de chaque ligne. "iterrows" permet de parcourir les lignes d'un DataFrame, mais il ne faut
pas le modifer lorsqu'on le parcourt.

Il faut donc créer un nouveau DataFrame et l'enregistrer à chaque itération.

Si le programme plante, on pourra ainsi récupérer le travail déjà effectué.

In [ ]:
df = pd.read_csv(PATH_TO_SECRETS + 'participants.csv')
d_temp = pd.DataFrame(columns=['Email', 'User group'])

for row in df.iterrows():
    # iterrows retourne des "Tuple" de type (<numéro_ligne>, {<colonne_B>: <valeur>})
    email = row[1]['Email']
    user_group = get_user_group_from_email(email)
    d_temp.loc[len(d_temp)] = [email, user_group]
    d_temp.to_csv('Resultat/participants_avec_user_group.csv')

In [ ]:
# Dans ce cas précis, cela ne fait pas de sens, mais ensuite l'idée serait d'utiliser "merge" pour fusionner
# les deux tables sur la base d'un identifiant unique
d_temp = d_temp.loc[:3]
d_temp

In [ ]:
# Variante pour reprendre un processus en cours
df = pd.read_csv(PATH_TO_SECRETS + 'participants.csv')

# Au moment de l'enregistrement en csv, on a laissé l'index. Il faut donc mettre
# la première colonne en tant qu'index.
d_temp = pd.read_csv('Resultat/participants_avec_user_group.csv', index_col=0)
d_temp = d_temp.loc[:3]

for row in df.iterrows():
    
    # "continue" permet de passer directement à l'itération suivante
    # `row[0]` contient l'index de la ligne et `len(d_temp)` le nombre de lignes déjà traitées
    if row[0] < len(d_temp):
        print(f'{row[0] + 1} / {len(df)}: "{row[1]["Email"]}" passé')
        continue
    
    # Essentiel de faire des logs
    print(f'{row[0] + 1} / {len(df)}: traitement "{row[1]["Email"]}"')
    
    email = row[1]['Email']
    user_group = get_user_group_from_email(email)
    
    # Motif très important pour ajouter une nouvelle ligne
    # à un DataFrame. Cela fonctionne seulement si on a
    # laissé l'index avec le numéro des lignes (comme
    # c'est le cas par défaut). Dans le cas contraire, il
    # faut mettre à la place du rang une valeur unique
    # d'index
    d_temp.loc[len(d_temp)] = [email, user_group]
    
    # Enregistrement de la table des lignes traitées
    d_temp.to_csv('Resultat/participants_ave_user_group.csv')

In [ ]:
d_temp